# Planning and Design with Semantic Kernel

This notebook demonstrates how to create a planning agent using Semantic Kernel that can break down complex travel requests into structured subtasks. We'll use Pydantic models to ensure structured output and explore how to design agents that can delegate work to specialized sub-agents.

## Import the Needed Packages

We'll import all the necessary libraries including Semantic Kernel for agent creation, Pydantic for data validation, and Azure OpenAI for the language model service.

In [10]:
import json
import os

from dotenv import load_dotenv

from pydantic import BaseModel, ValidationError, Field
from typing import List

from azure.identity import DefaultAzureCredential

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import AzureChatPromptExecutionSettings
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

from semantic_kernel.functions import KernelArguments

## Setting up Azure OpenAI Connection

Configure the Azure OpenAI service using either API key authentication or Azure AD authentication. This will be used by our Semantic Kernel agents.

In [11]:
load_dotenv()

# Option 1: Using API Key (recommended for development)
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY")
)

# Option 2: Using Azure AD Authentication (uncomment to use)
# Create Azure credential 
credential = DefaultAzureCredential()

# Create a token provider function
def get_azure_ad_token():
    """Function to get Azure AD token for OpenAI."""
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

# chat_completion_service = AzureChatCompletion(
#     deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
#     endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-02-01"),
#     ad_token=get_azure_ad_token()

## Define Pydantic Data Models

Create structured data models using Pydantic to ensure our planning agent returns well-formatted, validated responses. These models define the structure for subtasks and travel plans.

In [ ]:
# Define the structure for individual subtasks
class SubTask(BaseModel):
    # Which specialized agent should handle this subtask
    assigned_agent: str = Field(
        description="The specific agent assigned to handle this subtask")
    # Detailed description of what needs to be accomplished
    task_details: str = Field(
        description="Detailed description of what needs to be done for this subtask")


# Define the overall structure for the complete travel plan
class TravelPlan(BaseModel):
    # The original user request that needs to be broken down
    main_task: str = Field(
        description="The overall travel request from the user")
    # Collection of all subtasks that make up the complete plan
    subtasks: List[SubTask] = Field(
        description="List of subtasks broken down from the main task, each assigned to a specialized agent")

## Create the Planning Agent

Set up a specialized planning agent that can analyze travel requests and break them down into specific subtasks. The agent is configured with knowledge about different specialized agents and their capabilities.

In [13]:
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = """You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general requests"""

# Create the prompt execution settings and configure the Pydantic model response format
settings = AzureChatPromptExecutionSettings(response_format=TravelPlan)

agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings) 
)

## Test the Planning Agent

Run the planning agent with a sample travel request to see how it breaks down the complex task into structured subtasks assigned to different specialized agents.

In [14]:
from IPython.display import display, HTML


async def main():
    # Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    # Respond to user input
    user_inputs = [
        "Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne",
    ]

    for user_input in user_inputs:
        
        # Start building HTML output
        html_output = "<div style='margin-bottom:10px'>"
        html_output += "<div style='font-weight:bold'>User:</div>"
        html_output += f"<div style='margin-left:20px'>{user_input}</div>"
        html_output += "</div>"

        # Collect the agent's response
        response = await agent.get_response(messages=user_input, thread=thread)
        thread = response.thread

        try:
            # Try to validate the response as a TravelPlan
            travel_plan = TravelPlan.model_validate(json.loads(response.message.content))

            # Display the validated model as formatted JSON
            formatted_json = travel_plan.model_dump_json(indent=4)
            html_output += "<div style='margin-bottom:20px'>"
            html_output += "<div style='font-weight:bold'>Validated Travel Plan:</div>"
            html_output += f"<pre style='margin-left:20px; padding:10px; border-radius:5px;'>{formatted_json}</pre>"
            html_output += "</div>"
        except ValidationError as e:
            # Handle validation errors
            html_output += "<div style='margin-bottom:20px; color:red;'>"
            html_output += "<div style='font-weight:bold'>Validation Error:</div>"
            html_output += f"<pre style='margin-left:20px;'>{str(e)}</pre>"
            html_output += "</div>"
            # Add this to see what the response contains for debugging
            html_output += "<div style='margin-bottom:20px;'>"
            html_output += "<div style='font-weight:bold'>Raw Response:</div>"
            html_output += f"<div style='margin-left:20px; white-space:pre-wrap'>{response.content}</div>"
            html_output += "</div>"

        html_output += "<hr>"

        # Display formatted HTML
        display(HTML(html_output))

await main()

## Expected Output

The planning agent will generate a structured travel plan with subtasks assigned to specialized agents. Here's what you should expect to see:

You should see sample output similar to:

```json
User:
Create a travel plan for a family of 4, with 2 kids, from Singapore to Melboune
Validated Travel Plan:
{
    "main_task": "Plan a family trip from Singapore to Melbourne for 4 people including 2 kids.",
    "subtasks": [
        {
            "assigned_agent": "FlightBooking",
            "task_details": "Book round-trip flights from Singapore to Melbourne for 2 adults and 2 children."
        },
        {
            "assigned_agent": "HotelBooking",
            "task_details": "Find and book a family-friendly hotel in Melbourne that accommodates 4 people."
        },
        {
            "assigned_agent": "CarRental",
            "task_details": "Arrange for a car rental in Melbourne suitable for a family of 4."
        },
        {
            "assigned_agent": "ActivitiesBooking",
            "task_details": "Plan and book family-friendly activities in Melbourne suitable for kids."
        },
        {
            "assigned_agent": "DestinationInfo",
            "task_details": "Provide information about Melbourne, including attractions, dining options, and family-oriented activities."
        }
    ]
}
```

## Experiment with Different Travel Requests

Try modifying the user input in the code above to test how the planning agent handles different types of travel scenarios. You can experiment with different destinations, group sizes, and travel preferences.

In [ ]:
# Example: Try different travel scenarios
# Modify the user_inputs list in the main() function above with these examples:

example_requests = [
    "Plan a business trip for 2 people from New York to Tokyo for a tech conference",
    "Create a romantic getaway for 2 from Paris to Santorini for a honeymoon",
    "Organize a group adventure for 8 friends from London to Iceland for hiking and photography",
    "Plan a solo backpacking trip from Bangkok to various Southeast Asian countries",
    "Arrange a multi-generational family vacation for 12 people from Toronto to Disney World"
]

# Replace the user_inputs in the main() function with any of these examples
# Then run the main() function again to see how the planning agent adapts

print("Try these example requests by modifying the user_inputs list in the main() function above:")
for i, request in enumerate(example_requests, 1):
    print(f"{i}. {request}")

## Key Concepts Demonstrated

This notebook showcases several important AI agent design patterns:

### 1. **Structured Output with Pydantic**
- Using Pydantic models to ensure the agent returns well-formatted, validated data
- The `response_format` parameter enforces the agent to follow the defined schema

### 2. **Planning and Delegation**
- A planning agent that analyzes complex requests and breaks them into manageable subtasks
- Each subtask is assigned to a specialized agent based on its capabilities

### 3. **Agent Specialization**
- Different agents handle specific domains (flights, hotels, car rentals, activities, destination info)
- This modular approach allows for better expertise and easier maintenance

### 4. **Validation and Error Handling**
- The code includes proper validation to ensure the agent's response matches expected format
- Error handling provides debugging information when validation fails

This pattern is particularly useful for complex workflows that require coordination between multiple specialized systems or agents.